# Basic Capacioty Expansion Practical

In this practical, we aim to build a simple capacity expansion model for hydrogen supoply chain. The code is developed in python, and the demand, fuel price, network data, and zone characterisitcs are given.

### 1. Load and Process the data

Let's start by loading the data we have. Load the data from 'Ex03' Folder.

In [3]:
# Import the packages first

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB

In [19]:
# we save the adress to the data directory in a variable, so we can use it later
data_directory = "/workspaces/Supply_Chain_Analytics_2026/Exercise_Files/Ex03"

Fuels = pd.read_csv(f"{data_directory}/Fuels_data.csv")
Gen_data = pd.read_csv(f"{data_directory}/HSC_Gen_Data.csv")
Load = pd.read_csv(f"{data_directory}/HSC_load.csv")
Network = pd.read_csv(f"{data_directory}/HSC_Pipelines.csv")
Zone_data = pd.read_csv(f"{data_directory}/Zone_Char.csv")


In [20]:
# let's have a look at the data -- Remove the comment tags to show the data
Fuels.head()
#Gen_data.head()
#Load.head()
#Network.head()
#Zone_data.head()

# you can also use .describe() to get a statistical summary of the data
Load.describe()

,Time_Index,Load_HSC_Tonne_z1,Load_HSC_Tonee_z2,Load_HSC_Tonne_z3,Load_HSC_Tonne_z4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
count,8760.000000,8760.000000,8760.000000,8760.000000,8760.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,4380.500000,17.906473,8.635753,6.222432,12.485205,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2528.938512,8.924145,4.265556,3.142557,4.797066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,4.200000,2.400000,1.400000,5.300000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,2190.750000,11.200000,5.300000,3.800000,8.700000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,4380.500000,17.000000,8.200000,5.900000,11.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,6570.250000,23.500000,11.400000,8.200000,15.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,8760.000000,58.000000,26.700000,20.300000,32.100000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We need to define the set of generators, storage units, pipelines, and timesteps, to be able to define variables and constraints by them. 

In [21]:
# Let G represent set of all generators in the model.
# in the HSC_Gen_Data, the column 'H_Gen_type' indicates whether a resource is a generator (>0) or storage unit (0). 
# If it is 1, it is a generator that procudes emission suh SMR, if it is 2, it is a generator that produces zero-emission hydrogen via electrolysis.
G = Gen_data[Gen_data['H_Gen_type']>0]
#print(G['Resource'])

# Let S represent set of all storage units in the model.
S = Gen_data[Gen_data['H_Gen_type']==0]
#print(S['Resource'])

# Let I represent set of all pipelines in the model.
I = Network['HSC_Pipelines']
#print(I)

# Let T represent set of all timesteps in the model
T = Load['Time_Index']

# Set of all zones is represented by Z
Z = Zone_data['Zones']


### 2. Create the Model

In [22]:
hsc = gp.Model("HSC_Capacity_Expansion")

### 3. Define the Variables

The variables in the model are either related to Capacity, Operation, or Policy. It is common in programming that the variables are named with "camelCase" for better readability. In this way, we start a variable name with lowercase 'v', and each new word starts with capital letter.

In [ ]:
# Defining the Capacity decisions variables

# New and Retired Generation Capacity variables
vNewGenCap = hsc.addVars(G['r_id'], name="NewGenCap", lb=0, vtype=GRB.INTEGER)
vRetGenCap = hsc.addVars(G['r_id'], name="RetGenCap", lb=0, vtype=GRB.INTEGER)

# New and Retired Storage Capacity variables
vNewStorCap = hsc.addVars(S['r_id'], name="NewStorCap", lb=0, vtype=GRB.INTEGER)
vRetStorCap = hsc.addVars(S['r_id'], name="RetStorCap", lb=0, vtype=GRB.INTEGER)

# New and Retired Pipeline Capacity variables
vNewPipe = hsc.addVars(I, name="NewPipeCap", lb=0, vtype=GRB.INTEGER)
vRetPipe = hsc.addVars(I, name="RetPipeCap", lb=0, vtype=GRB.INTEGER)

######## .......................#############
# Defining the Operation decision variables #
######## .......................############

# Generation from generators for each generator and timestep
vGen = hsc.addVars(G['r_id'], T, name="Gen", lb=0, vtype=GRB.CONTINUOUS) 

# Storage charge and discharge for each storage unit and timestep
vStorCharge = hsc.addVars(S['r_id'], T, name="StorCharge", lb=0, vtype=GRB.CONTINUOUS) 
vStorDischarge = hsc.addVars(S['r_id'], T, name="StorDischarge", lb=0, vtype=GRB.CONTINUOUS)
# State of Charge for each storage unit and timestep
vStorSOC = hsc.addVars(S['r_id'], T, name="StorSOC", lb=0, vtype=GRB.CONTINUOUS)

# Pipeline flow for each pipeline and timestep
vPipeFlow = hsc.addVars(I, T, name="PipeFlow", vtype=GRB.CONTINUOUS) # Note: can be negative for bi-directional flow



